In [4]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import pickle
from data_mnist.mnist import load_mnist
from common.function import sigmoid, softmax
from PIL import Image

## 모듈 가져오기
- 기본 모듈 가져오기 (sys, os, numpy, pickle, PIL)
- 만들어 둔 함수를 가져오기 (sigmoid, softmax)
- mnist 자료 가져오기 (mnist.py 안에 있는 load_mnist 함수)
- github 에서 function.py, mnist.py, sample_weight.pkl 가져오기
   * 현재 디렉토리 아래에 \common\ 디렉토리 만들고 **"function.py"** 저장
   * 현재 디렉토리 아래에 \data_mnist\ 디렉토리 만들고 **"mnist.py"** 저장
   * 현재 디렉토리에 **"sample_weight.pkl"** 저장
   * https://github.com/eventia/deepLearningOpenLecture/tree/master/Step07_MNIST
   * https://goo.gl/p8F24N

In [7]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

### mnist 가져오기
#### 학습할 자료
- x_train, t_train

#### 검증(테스트)할 자료
- x_test, t_test

In [8]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)


(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [10]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
img = x_train[35]
label = t_train[35]
print(label)

print(img.shape)  # 784
img = img.reshape(28,28)
print(img.shape)  # 28,28
img_show(img)


5
(784,)
(28, 28)


In [15]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    return y

In [26]:
x,t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1
print("Accuracy:" + str(float(accuracy_cnt/len(x))))
    

Accuracy:0.9352


In [19]:
x,_ = get_data()
network = init_network()
W1,W2,W3 = network['W1'], network['W2'], network['W3']

In [21]:
print(x.shape, W1.shape, W2.shape, W3.shape, y.shape)
print(x[0].shape, W1.shape, W2.shape, W3.shape, y.shape)

(10000, 784) (784, 50) (50, 100) (100, 10) (10,)
(784,) (784, 50) (50, 100) (100, 10) (10,)


\begin{matrix}
10000번 반복 - 1개 : &X   &W1           &W2 &W3  &\Longrightarrow &Y        \\
&784   &784 \times 50   &50 \times 100 &100 \times 10  &\Longrightarrow  &10  \\\\
1번 - 10000개 : &X   &W1           &W2 &W3  &\Longrightarrow &Y        \\
&10000 \times 784   &784 \times 50   &50 \times 100 &100 \times 10  &\Longrightarrow  &10000 \times 10  \\\\
M번 반복 - N개 : &X   &W1           &W2 &W3  &\Longrightarrow &Y        \\
&N \times 784   &784 \times 50   &50 \times 100 &100 \times 10  &\Longrightarrow  &N \times 10  \\
\end{matrix}

## 배치처리
- IO (입력 Input, 출력 Output) 에 상대적으로 많은 시간 소요
- 메모리가 허용하는 한 한번에 많은 데이터를 읽어옴
- 1장씩 사진을 처리하면 1장씩 10000번을 읽어와야 하지만
- 100장씩 사진을 처리하면 100장씩 100번을 읽어오면 됨 
- 즉, 1장씩 1만번 읽기를 반복하는건 보다 100장씩 100번 반복에 걸리는 시간이 짧음
- 큰 배열을 계산하는 것이 작은 배열을 여러번 계산하는 것보다 빠름

In [25]:
x,t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0,len(x),batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p==t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt/len(x))))

Accuracy:0.9352
